<a href="https://colab.research.google.com/github/nnilayy/Unet/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt 
import tensorflow as tf 
import numpy as np
import cv2 
import os 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir "/content/drive/MyDrive/dataset"

In [ ]:
!unzip -q /content/drive/MyDrive/dataset.zip -d /content/drive/MyDrive/

In [4]:
train=ImageDataGenerator(rescale=1/255,
                         rotation_range=40,
                         brightness_range=(0,5),
                         zoom_range=0.2,
                         shear_range=0.2,
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode="nearest",
                         )

validation=ImageDataGenerator(rescale=1/255,
                              rotation_range=40,
                              brightness_range=(0,5),
                              zoom_range=0.2,
                              shear_range=0.2,
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode="nearest",)

In [6]:
train_dataset=train.flow_from_directory('/content/drive/MyDrive/dataset/train',
                                        target_size=(130,130),
                                        batch_size=3,
                                        class_mode='binary')
val_dataset=validation.flow_from_directory('/content/drive/MyDrive/dataset/validation',
                                        target_size=(130,130),
                                        batch_size=3,
                                        class_mode='binary')

Found 301 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


In [ ]:
train_dataset.class_indices

{'Cracked': 0, 'Not Cracked': 1}

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dropout, BatchNormalization, InputLayer, Reshape

model = Sequential()
model.add(InputLayer(input_shape=(130, 130, 3)))
model.add(Conv2D(16, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(16, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(8, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(8, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Flatten())
model.add(Dense(1, activation="sigmoid"))
model.summary()






# ---------------------------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------
# (1)Params: 32,000
# (2)Train_Accuracy: 85.45
# (3)Val_Accuracy: 84.44
# model = Sequential()
# model.add(InputLayer(input_shape=(224, 224,3)))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Flatten())
# model.add(Dense(32, activation="relu"))
# model.add(Dense(1, activation="sigmoid"))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 130, 130, 16)      448       
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 65, 65, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 65, 65, 16)        2320      
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 33, 33, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_32 (Conv2D)          (None, 33, 33, 8)         1160      
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 17, 17, 8)       

In [ ]:
model.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3), 
              metrics=["accuracy"])
history=model.fit(train_dataset, steps_per_epoch=16, epochs=100, validation_data=val_dataset)

Epoch 1/100
16/16 [==============================] - 3s 80ms/step - loss: 0.6941 - accuracy: 0.5208 - val_loss: 0.6931 - val_accuracy: 0.5222
Epoch 2/100
16/16 [==============================] - 1s 73ms/step - loss: 0.6505 - accuracy: 0.7292 - val_loss: 0.9004 - val_accuracy: 0.4444
Epoch 3/100
16/16 [==============================] - 1s 72ms/step - loss: 0.7722 - accuracy: 0.4783 - val_loss: 0.7069 - val_accuracy: 0.5556
Epoch 4/100
16/16 [==============================] - 1s 72ms/step - loss: 0.7150 - accuracy: 0.3958 - val_loss: 0.6946 - val_accuracy: 0.4444
Epoch 5/100
16/16 [==============================] - 1s 72ms/step - loss: 0.6872 - accuracy: 0.6458 - val_loss: 0.6802 - val_accuracy: 0.5556
Epoch 6/100
16/16 [==============================] - 1s 72ms/step - loss: 0.6909 - accuracy: 0.4783 - val_loss: 0.6869 - val_accuracy: 0.6444
Epoch 7/100
16/16 [==============================] - 1s 77ms/step - loss: 0.6892 - accuracy: 0.6250 - val_loss: 0.7040 - val_accuracy: 0.4667
Epoch 

In [31]:
model.save("/content/drive/MyDrive/model_130_2.h5")

In [ ]:
file="/content/images.jpg"
image=cv2.imread(file)
image=cv2.resize(image,(224,224))
image=np.expand_dims(image,0)
prediction=int(model.predict(image))

if prediction == 0:
  print('Cracked')

elif prediction==1:
  print('Not Cracked')

In [ ]:
# https://www.youtube.com/watch?v=uqomO_BZ44g

In [ ]:
import os
def get_file_size(file_path):
  size=os.path.getsize(file_path)
  return size

In [ ]:
def convert_bytes(size,unit=None):
  if unit=="KB":
    return print("File size: "+str(round(size/1024,3))+" Kilobytes")
  elif unit=="MB":
    return print("File size: "+str(round(size/(1024*1024),3))+" Megabytes")
  else:
    return print("File size: "+str(size)+"bytes")

In [ ]:
model="/content/model_500.h5"

In [ ]:
# Model Size Before Model Quantization
convert_bytes(get_file_size(model),"MB")

File size: 0.068 Megabytes


In [ ]:
import tensorflow as tf
model=tf.keras.models.load_model(model)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations=[tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter = True
tflite_model_name="model.tflite"
tflite_model = converter.convert()
open(tflite_model_name,"wb").write(tflite_model)

16000

In [ ]:
# Model Size After Model Quantization
tflite_model="/content/model.tflite"
convert_bytes(get_file_size(tflite_model),"KB")

File size: 15.625 Kilobytes


In [ ]:
# ()Dataset 
# ()Inferencing Code on Arduino
# ()SD Card model transfer
# ()Library Creation and function deletion

In [ ]:
import binascii
def convert_to_c_array(bytes) -> str:
  hexstr = binascii.hexlify(bytes).decode("UTF-8")
  hexstr = hexstr.upper()
  array = ["0x" + hexstr[i:i + 2] for i in range(0, len(hexstr), 2)]
  array = [array[i:i+10] for i in range(0, len(array), 10)]
  return ",\n  ".join([", ".join(e) for e in array])

In [ ]:
tflite_binary = open('model.tflite', 'rb').read()
ascii_bytes = convert_to_c_array(tflite_binary)
header_file = "const unsigned char model_tflite[] = {\n  " + ascii_bytes + "\n};\nunsigned int model_tflite_len = " + str(len(tflite_binary)) + ";"
open("model3.h", "w").write(header_file)
convert_bytes(get_file_size('/content/model3.h'),"KB")